# Import all the libraries here

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fredapi import Fred
import yfinance as yf
from datetime import datetime
%matplotlib inline
from scipy.stats import chi2_contingency

pd.set_option('display.max_columns', None)
# API keys
f = open("../fred_api_key.dat", "r")
fred_api = f.read()

# Get your data here

In [2]:
fred = Fred(api_key=fred_api)

In [3]:
def get_ticker_data(ticker_symbol, plot=False):
    data = yf.Ticker(ticker_symbol)
    ticker = data.history(period='max')
    ticker.drop(columns=['Dividends', 'Stock Splits'], inplace=True)
    ticker.reset_index(inplace=True)

    for i in range(len(ticker['Date'])):
        ticker.iloc[i, 0] = ticker.iloc[i, 0].date()

    if plot:
        plt.plot(np.arange(0, len(ticker['Date'])), ticker['Open'])
        plt.show()

    return ticker

In [4]:
gdp = fred.get_series("GDP") # gdp of US
inflation = fred.get_series("CPIAUCSL") # CPI
unemployment = fred.get_series("UNRATE") # unemployment rate
fed_int_rate = fred.get_series("FEDFUNDS") # federal funds rate
ten_y_tres_yield = fred.get_series("DGS10") # US 10 year bond yield
m2_mon_supp = fred.get_series("M2SL") # how much money is circulationg in the economy
cons_sent_idx = fred.get_series("UMCSENT") # measures consumer confidence
ind_pro_idx = fred.get_series("INDPRO") # industrial production, total index
exports = fred.get_series("EXPGS") # exports of goods and services
imports = fred.get_series("IMPGS") # imports of goods and services
new_home_const = fred.get_series("HOUST") # new privately owned housing units
mortgage_rate = fred.get_series("MORTGAGE30US") # 30-Year Fixed Rate Mortgage Average
volatility_index = fred.get_series("VIXCLS") # CBOE Volatility Index
crude_oil_wti = fred.get_series("DCOILWTICO") # crude oil price, wti

# Know your data here

In [5]:
gdp

1946-01-01          NaN
1946-04-01          NaN
1946-07-01          NaN
1946-10-01          NaN
1947-01-01      243.164
                ...    
2023-10-01    28296.967
2024-01-01    28624.069
2024-04-01    29016.714
2024-07-01    29374.914
2024-10-01    29719.647
Length: 316, dtype: float64

In [6]:
inflation

1947-01-01     21.480
1947-02-01     21.620
1947-03-01     22.000
1947-04-01     22.000
1947-05-01     21.950
               ...   
2024-10-01    315.564
2024-11-01    316.449
2024-12-01    317.603
2025-01-01    319.086
2025-02-01    319.775
Length: 938, dtype: float64

# Create dataframes here

In [7]:
df_gdp = gdp.to_frame(name="gdp").reset_index().rename(columns={"index" : "date"})
df_infl = inflation.to_frame(name="inflation").reset_index().rename(columns={"index" : "date"})
df_unem = unemployment.to_frame(name="unemployment").reset_index().rename(columns={"index" : "date"})
df_fed_rate = fed_int_rate.to_frame(name="fed_int_rate").reset_index().rename(columns={"index" : "date"})
df_ten_y_yield = ten_y_tres_yield.to_frame(name="ten_year_yield").reset_index().rename(columns={"index" : "date"})
df_m2 = m2_mon_supp.to_frame(name="m2_money_supp").reset_index().rename(columns={"index" : "date"})
df_consum_sent = cons_sent_idx.to_frame(name="consum_sent_idx").reset_index().rename(columns={"index" : "date"})
df_ind_pro = ind_pro_idx.to_frame(name="indus_pro_idx").reset_index().rename(columns={"index" : "date"})
df_exports = exports.to_frame(name="exports").reset_index().rename(columns={"index" : "date"})
df_imports = imports.to_frame(name="imports").reset_index().rename(columns={"index" : "date"})
df_new_home = new_home_const.to_frame(name="new_home_const").reset_index().rename(columns={"index" : "date"})
df_mortgage_rate = mortgage_rate.to_frame(name="mortgage_rate").reset_index().rename(columns={"index" : "date"})
df_volatility_idx = volatility_index.to_frame(name="vix").reset_index().rename(columns={"index" : "date"})
df_wti = crude_oil_wti.to_frame(name="wti").reset_index().rename(columns={"index" : "date"})

In [8]:
df_gdp

,date,gdp
0,1946-01-01,NaN
1,1946-04-01,NaN
2,1946-07-01,NaN
3,1946-10-01,NaN
4,1947-01-01,243.164
...,...,...
311,2023-10-01,28296.967
312,2024-01-01,28624.069
313,2024-04-01,29016.714
314,2024-07-01,29374.914


In [9]:
data = [df_gdp, df_fed_rate, df_ten_y_yield, df_m2, df_consum_sent, df_ind_pro, 
        df_exports, df_imports, df_new_home, df_mortgage_rate, df_volatility_idx, 
        df_wti]

df_merged = df_infl.merge(df_unem, how="outer", on="date")

for i in data:
    df_merged = df_merged.merge(i, how="outer", on="date")

In [10]:
df_merged

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti
0,1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8654,NaN,NaN,NaN,NaN,NaN,NaN
1,1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6504,NaN,NaN,NaN,NaN,NaN,NaN
2,1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5160,NaN,NaN,NaN,NaN,NaN,NaN
3,1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5966,NaN,NaN,NaN,NaN,NaN,NaN
4,1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6235,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17225,2025-03-18,NaN,NaN,NaN,NaN,4.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.70,NaN
17226,2025-03-19,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.90,NaN
17227,2025-03-20,NaN,NaN,NaN,NaN,4.24,NaN,NaN,NaN,NaN,NaN,NaN,6.67,19.80,NaN
17228,2025-03-21,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.28,NaN


In [11]:
df_merged.isnull().sum()

date                   0
inflation          16292
unemployment       16304
gdp                16918
fed_int_rate       16382
ten_year_yield      1440
m2_money_supp      16437
consum_sent_idx    16573
indus_pro_idx      15956
exports            16918
imports            16918
new_home_const     16436
mortgage_rate      14413
vix                 8338
wti                 7359
dtype: int64

## Get S&P 500 data

In [12]:
sp500 = get_ticker_data("^SPX", plot=False)

/tmp/ipykernel_37253/3764776260.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1927-12-30' has dtype incompatible with datetime64[ns, America/New_York], please explicitly cast to a compatible dtype first.
  ticker.iloc[i, 0] = ticker.iloc[i, 0].date()


In [13]:
sp500

,Date,Open,High,Low,Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,0
...,...,...,...,...,...,...
24418,2025-03-19,5632.370117,5715.330078,5622.200195,5675.290039,4660090000
24419,2025-03-20,5646.919922,5711.149902,5632.330078,5662.890137,4678420000
24420,2025-03-21,5630.729980,5670.839844,5603.100098,5667.560059,9367460000
24421,2025-03-24,5718.080078,5775.140137,5718.080078,5767.569824,4517990000


In [14]:
sp500.rename(columns={"Date": "date", "Close": "sp_close"}, inplace=True)

In [15]:
sp500["date"] = pd.to_datetime(sp500["date"])

In [16]:
df_merged = df_merged.merge(sp500, on="date", how="outer")
df_merged

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti,Open,High,Low,sp_close,Volume
0,1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25475,2025-03-19,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.90,NaN,5632.370117,5715.330078,5622.200195,5675.290039,4.660090e+09
25476,2025-03-20,NaN,NaN,NaN,NaN,4.24,NaN,NaN,NaN,NaN,NaN,NaN,6.67,19.80,NaN,5646.919922,5711.149902,5632.330078,5662.890137,4.678420e+09
25477,2025-03-21,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.28,NaN,5630.729980,5670.839844,5603.100098,5667.560059,9.367460e+09
25478,2025-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,NaN,5718.080078,5775.140137,5718.080078,5767.569824,4.517990e+09


In [17]:
# drop open, high, low
df_merged.drop(columns=["Open", "High", "Low"], inplace=True)

In [18]:
df_merged

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti,sp_close,Volume
0,1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25475,2025-03-19,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.90,NaN,5675.290039,4.660090e+09
25476,2025-03-20,NaN,NaN,NaN,NaN,4.24,NaN,NaN,NaN,NaN,NaN,NaN,6.67,19.80,NaN,5662.890137,4.678420e+09
25477,2025-03-21,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.28,NaN,5667.560059,9.367460e+09
25478,2025-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,NaN,5767.569824,4.517990e+09


In [19]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             25480 non-null  datetime64[ns]
 1   inflation        938 non-null    float64       
 2   unemployment     926 non-null    float64       
 3   gdp              312 non-null    float64       
 4   fed_int_rate     848 non-null    float64       
 5   ten_year_yield   15790 non-null  float64       
 6   m2_money_supp    793 non-null    float64       
 7   consum_sent_idx  657 non-null    float64       
 8   indus_pro_idx    1274 non-null   float64       
 9   exports          312 non-null    float64       
 10  imports          312 non-null    float64       
 11  new_home_const   794 non-null    float64       
 12  mortgage_rate    2817 non-null   float64       
 13  vix              8892 non-null   float64       
 14  wti              9871 non-null   float

In [20]:
df = df_merged.copy()

# Clean your data here

## Missing value imputation

In [21]:
df

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti,sp_close,Volume
0,1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25475,2025-03-19,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.90,NaN,5675.290039,4.660090e+09
25476,2025-03-20,NaN,NaN,NaN,NaN,4.24,NaN,NaN,NaN,NaN,NaN,NaN,6.67,19.80,NaN,5662.890137,4.678420e+09
25477,2025-03-21,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.28,NaN,5667.560059,9.367460e+09
25478,2025-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,NaN,5767.569824,4.517990e+09


In [22]:
df = df[df['date'] > '2000-01-01']

In [23]:
df = df.reset_index(drop=True)

In [24]:
df

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti,sp_close,Volume
0,2000-01-03,NaN,NaN,NaN,NaN,6.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.21,NaN,1455.219971,9.318000e+08
1,2000-01-04,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.01,25.56,1399.420044,1.009000e+09
2,2000-01-05,NaN,NaN,NaN,NaN,6.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.41,24.65,1402.109985,1.085500e+09
3,2000-01-06,NaN,NaN,NaN,NaN,6.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.73,24.79,1403.449951,1.092300e+09
4,2000-01-07,NaN,NaN,NaN,NaN,6.52,NaN,NaN,NaN,NaN,NaN,NaN,8.15,21.72,24.79,1441.469971,1.225200e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6663,2025-03-19,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.90,NaN,5675.290039,4.660090e+09
6664,2025-03-20,NaN,NaN,NaN,NaN,4.24,NaN,NaN,NaN,NaN,NaN,NaN,6.67,19.80,NaN,5662.890137,4.678420e+09
6665,2025-03-21,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.28,NaN,5667.560059,9.367460e+09
6666,2025-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,NaN,5767.569824,4.517990e+09


In [25]:
# fill NaN values for sp_close with previous values
df["sp_close"] = df["sp_close"].ffill()
df["vix"] = df["vix"].ffill()
df["wti"] = df["wti"].ffill()
df["Volume"] = df["Volume"].ffill()
df["ten_year_yield"] = df["ten_year_yield"].ffill()

In [26]:
df

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti,sp_close,Volume
0,2000-01-03,NaN,NaN,NaN,NaN,6.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.21,NaN,1455.219971,9.318000e+08
1,2000-01-04,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.01,25.56,1399.420044,1.009000e+09
2,2000-01-05,NaN,NaN,NaN,NaN,6.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.41,24.65,1402.109985,1.085500e+09
3,2000-01-06,NaN,NaN,NaN,NaN,6.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.73,24.79,1403.449951,1.092300e+09
4,2000-01-07,NaN,NaN,NaN,NaN,6.52,NaN,NaN,NaN,NaN,NaN,NaN,8.15,21.72,24.79,1441.469971,1.225200e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6663,2025-03-19,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.90,67.84,5675.290039,4.660090e+09
6664,2025-03-20,NaN,NaN,NaN,NaN,4.24,NaN,NaN,NaN,NaN,NaN,NaN,6.67,19.80,67.84,5662.890137,4.678420e+09
6665,2025-03-21,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.28,67.84,5667.560059,9.367460e+09
6666,2025-03-24,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,67.84,5767.569824,4.517990e+09


In [27]:
df

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti,sp_close,Volume
0,2000-01-03,NaN,NaN,NaN,NaN,6.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.21,NaN,1455.219971,9.318000e+08
1,2000-01-04,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.01,25.56,1399.420044,1.009000e+09
2,2000-01-05,NaN,NaN,NaN,NaN,6.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.41,24.65,1402.109985,1.085500e+09
3,2000-01-06,NaN,NaN,NaN,NaN,6.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.73,24.79,1403.449951,1.092300e+09
4,2000-01-07,NaN,NaN,NaN,NaN,6.52,NaN,NaN,NaN,NaN,NaN,NaN,8.15,21.72,24.79,1441.469971,1.225200e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6663,2025-03-19,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.90,67.84,5675.290039,4.660090e+09
6664,2025-03-20,NaN,NaN,NaN,NaN,4.24,NaN,NaN,NaN,NaN,NaN,NaN,6.67,19.80,67.84,5662.890137,4.678420e+09
6665,2025-03-21,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.28,67.84,5667.560059,9.367460e+09
6666,2025-03-24,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.48,67.84,5767.569824,4.517990e+09


In [28]:
# ffill all the other columns
df["inflation"] = df["inflation"].ffill()
df["unemployment"] = df["unemployment"].ffill()
df["gdp"] = df["gdp"].ffill()
df["fed_int_rate"] = df["fed_int_rate"].ffill()
df["m2_money_supp"] = df["m2_money_supp"].ffill()
df["consum_sent_idx"] = df["consum_sent_idx"].ffill()
df["indus_pro_idx"] = df["indus_pro_idx"].ffill()
df["exports"] = df["exports"].ffill()
df["imports"] = df["imports"].ffill()
df["new_home_const"] = df["new_home_const"].ffill()
df["mortgage_rate"] = df["mortgage_rate"].ffill()

In [29]:
df

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti,sp_close,Volume
0,2000-01-03,NaN,NaN,NaN,NaN,6.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.21,NaN,1455.219971,9.318000e+08
1,2000-01-04,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.01,25.56,1399.420044,1.009000e+09
2,2000-01-05,NaN,NaN,NaN,NaN,6.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.41,24.65,1402.109985,1.085500e+09
3,2000-01-06,NaN,NaN,NaN,NaN,6.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.73,24.79,1403.449951,1.092300e+09
4,2000-01-07,NaN,NaN,NaN,NaN,6.52,NaN,NaN,NaN,NaN,NaN,NaN,8.15,21.72,24.79,1441.469971,1.225200e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6663,2025-03-19,319.775,4.1,29719.647,4.33,4.25,21561.4,71.7,104.2062,3218.6,4149.659,1501.0,6.65,19.90,67.84,5675.290039,4.660090e+09
6664,2025-03-20,319.775,4.1,29719.647,4.33,4.24,21561.4,71.7,104.2062,3218.6,4149.659,1501.0,6.67,19.80,67.84,5662.890137,4.678420e+09
6665,2025-03-21,319.775,4.1,29719.647,4.33,4.25,21561.4,71.7,104.2062,3218.6,4149.659,1501.0,6.67,19.28,67.84,5667.560059,9.367460e+09
6666,2025-03-24,319.775,4.1,29719.647,4.33,4.25,21561.4,71.7,104.2062,3218.6,4149.659,1501.0,6.67,17.48,67.84,5767.569824,4.517990e+09


In [30]:
df.isnull().sum()

date                0
inflation          21
unemployment       21
gdp                65
fed_int_rate       21
ten_year_yield      0
m2_money_supp      21
consum_sent_idx    21
indus_pro_idx      21
exports            65
imports            65
new_home_const     21
mortgage_rate       4
vix                 0
wti                 1
sp_close            0
Volume              0
dtype: int64

In [32]:
df = df.dropna()

In [34]:
df = df.reset_index(drop=True)

In [35]:
df

,date,inflation,unemployment,gdp,fed_int_rate,ten_year_yield,m2_money_supp,consum_sent_idx,indus_pro_idx,exports,imports,new_home_const,mortgage_rate,vix,wti,sp_close,Volume
0,2000-04-01,170.900,3.8,10247.720,6.02,6.03,4767.8,109.2,92.6659,1093.36,1455.860,1626.0,8.23,24.11,26.86,1498.579956,1.227400e+09
1,2000-04-03,170.900,3.8,10247.720,6.02,6.00,4767.8,109.2,92.6659,1093.36,1455.860,1626.0,8.23,24.03,26.28,1505.969971,1.021700e+09
2,2000-04-04,170.900,3.8,10247.720,6.02,5.90,4767.8,109.2,92.6659,1093.36,1455.860,1626.0,8.23,27.12,25.46,1494.729980,1.515460e+09
3,2000-04-05,170.900,3.8,10247.720,6.02,5.90,4767.8,109.2,92.6659,1093.36,1455.860,1626.0,8.23,28.41,25.76,1487.369995,1.110300e+09
4,2000-04-06,170.900,3.8,10247.720,6.02,5.93,4767.8,109.2,92.6659,1093.36,1455.860,1626.0,8.23,27.15,25.51,1501.339966,1.008000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6598,2025-03-19,319.775,4.1,29719.647,4.33,4.25,21561.4,71.7,104.2062,3218.60,4149.659,1501.0,6.65,19.90,67.84,5675.290039,4.660090e+09
6599,2025-03-20,319.775,4.1,29719.647,4.33,4.24,21561.4,71.7,104.2062,3218.60,4149.659,1501.0,6.67,19.80,67.84,5662.890137,4.678420e+09
6600,2025-03-21,319.775,4.1,29719.647,4.33,4.25,21561.4,71.7,104.2062,3218.60,4149.659,1501.0,6.67,19.28,67.84,5667.560059,9.367460e+09
6601,2025-03-24,319.775,4.1,29719.647,4.33,4.25,21561.4,71.7,104.2062,3218.60,4149.659,1501.0,6.67,17.48,67.84,5767.569824,4.517990e+09


In [36]:
df.isnull().sum()

date               0
inflation          0
unemployment       0
gdp                0
fed_int_rate       0
ten_year_yield     0
m2_money_supp      0
consum_sent_idx    0
indus_pro_idx      0
exports            0
imports            0
new_home_const     0
mortgage_rate      0
vix                0
wti                0
sp_close           0
Volume             0
dtype: int64

In [38]:
# save the data
df.to_csv("../data/financial_data_clean.csv")